![Alt text](image-1.png)

In [304]:
# %pip install -r requirements.txt
# !pip install scikit-learn==1.2.2
# !pip install openpyxl

In [305]:
# namesheet = 1MPK, 2MKS, 3MPV, 4MPL, 5MAC, 6MPDC, 7SB, 8MKB

In [306]:
import pickle
import numpy as np
import pandas as pd

filename_all = 'model/finalized_model.pkl'
filename = 'model/M8.pkl'
namesheet = '8MKB'
outputFile= 'T2_&_All\DataM8.xlsx'     
loaded_model = pickle.load(open(filename, 'rb'))
loaded_model_all = pickle.load(open(filename_all, 'rb'))
df=pd.read_excel('CropCuttingCaneQuality_6667_forAIModel.xlsx', sheet_name=namesheet, parse_dates=True)


In [307]:
pd.__version__

'1.5.3'

In [308]:
# %pip install --upgrade pip

In [309]:

def fiber_T1(x):
    a,b,c,d=[  2.01379247, -20.78037567,   3.5155386,  11.00677066]
    g=a*np.log(c+(b/x)+(b/x**2))+d
    return g

def fiber_T2(x):
    a,b,c,d=[  2.59140136, -20.42160029,   3.9202099,  10.15451078]
    g=a*np.log(c+(b/x)+(b/x**2))+d
    return g

def fiber_T3(x):
    a,b,c,d=[12.9574611, -4.9670637,   2.73127967,  1.02561215]
    g=a*np.log(c+(b/x)+(b/x**2))+d
    return g

def fiber_T4(x):
    a,b,c,d=[  4.62564942, -17.53919804,   3.66075841,  10.03896733]
    g=a*np.log(c+(b/x)+(b/x**2))+d
    return g

def fiber_T5(x):
    a,b,c,d=[  1.02179087, -22.92415521,   3.80403583,  12.04063676]
    g=a*np.log(c+(b/x)+(b/x**2))+d
    return g

def fiber_T6(x):
    a,b,c,d=[  3.29687238, -19.58092598,   4.08787093,  10.41188387]
    g=a*np.log(c+(b/x)+(b/x**2))+d
    return g

In [310]:
df = pd.DataFrame(df)
df_temp = pd.DataFrame(df)

In [311]:
#For sheet name 7SB
# df = df[['วันที่ปลูก','BrixFleld','CCS']] #for 7SB
# df['วันที่ปลูก'] = pd.to_datetime(df['วันที่ปลูก'])#for 7SB
# df['month'] = df['วันที่ปลูก'].dt.month#for 7SB

In [312]:
#For other sheet name
df = df[['cane_ac_date','BrixFleld','CCS']]
df['cane_ac_date'] = pd.to_datetime(df['cane_ac_date'])
df['month'] = df['cane_ac_date'].dt.month
df['month']

0      3
1     11
2      3
3      6
4     12
5      1
6      2
7      4
8      3
9      3
10    12
11     5
12     2
13     1
14     5
15     1
16     1
17    11
18     4
19     6
20    12
21     1
22     3
23     6
24     1
25     1
26     3
27     5
28    12
29     1
30     3
31     5
32    10
33     3
34     3
35     6
Name: month, dtype: int64

In [313]:
 # def assign_fiber(row):
#     if 5 <= row['month'] <= 10:
#         return fiber_T2(row['month'])
#     elif row['month'] > 10 :
#         return fiber_T2(row['month'])
   
def change_month(month):
    if 1 <= month <= 6:
        return 7
    else:
        return month

# Apply the function to each row
df['month'] = df['month'].apply(change_month)

def assign_fiber(row):
    if 0 <= row['month']:
        return fiber_T2(row['month'])

# Apply the function to each row
df['Fb'] = df.apply(assign_fiber, axis=1)


In [314]:
# df.dropna(inplace=True)

In [315]:
test=pd.DataFrame({'BrixF':df['BrixFleld'],'Fiber':df['Fb']})

In [316]:
result = loaded_model.predict(test)
df['CCSpre'] = result

In [317]:
result_all = loaded_model_all.predict(test)
df['CCSpre_all'] = result_all

In [318]:
actuleCCS = df['CCS']
preCCS = df['CCSpre']
preCCS_all = df['CCSpre_all']

In [319]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
mse = mean_squared_error(actuleCCS, preCCS)
mae = mean_absolute_error(actuleCCS, preCCS)
r2 = r2_score(actuleCCS, preCCS)
df['MSE'] = mse
df['MAE'] = mae
df['R2'] = r2

In [320]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
mse = mean_squared_error(actuleCCS, preCCS_all)
mae = mean_absolute_error(actuleCCS, preCCS_all)
r2 = r2_score(actuleCCS, preCCS_all)
df['MSE_all'] = mse
df['MAE_all'] = mae
df['R2_all'] = r2

In [321]:
df

,cane_ac_date,BrixFleld,CCS,month,Fb,CCSpre,CCSpre_all,MSE,MAE,R2,MSE_all,MAE_all,R2_all
0,2023-03-01,19.200000,8.395512,7,8.769888,7.169960,6.684302,6.06494,2.136949,-0.19383,4.882342,1.804386,0.038954
1,2022-11-10,18.466667,10.724603,11,11.810884,8.077014,7.463874,6.06494,2.136949,-0.19383,4.882342,1.804386,0.038954
2,2023-03-08,17.266667,9.136375,7,8.769888,5.709565,6.835811,6.06494,2.136949,-0.19383,4.882342,1.804386,0.038954
3,2023-06-09,16.133333,5.777622,7,8.769888,5.698781,6.657150,6.06494,2.136949,-0.19383,4.882342,1.804386,0.038954
4,2022-12-18,21.133333,10.232160,12,12.048122,9.530526,9.192944,6.06494,2.136949,-0.19383,4.882342,1.804386,0.038954
5,2023-01-24,19.333333,9.478809,7,8.769888,7.169960,6.684302,6.06494,2.136949,-0.19383,4.882342,1.804386,0.038954
6,2023-02-14,22.866667,11.019793,7,8.769888,7.679492,11.233007,6.06494,2.136949,-0.19383,4.882342,1.804386,0.038954
7,2023-04-17,12.333333,3.971810,7,8.769888,5.228605,7.207810,6.06494,2.136949,-0.19383,4.882342,1.804386,0.038954
8,2023-03-01,21.533333,11.830416,7,8.769888,7.990328,10.624350,6.06494,2.136949,-0.19383,4.882342,1.804386,0.038954
9,2023-03-13,18.866667,10.450247,7,8.769888,7.169960,6.626047,6.06494,2.136949,-0.19383,4.882342,1.804386,0.038954


In [322]:
# with pd.ExcelWriter('DATAM1.xlsx', engine='openpyxl') as writer:
#     df.to_excel(writer, index=False)

# with pd.ExcelWriter('DATAM1.xlsx', engine='xlsxwriter') as writer:
#     df.to_excel(writer, index=False)


df.to_excel(outputFile, index=False)

